# Tavily Web Search
- LLM에게 웹검색을 쥐어주자

In [4]:
%pip install -q langchain-tavily

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

True

In [6]:
from langchain_tavily import TavilySearch   

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [9]:
pprint(search_tool.invoke({'query': '강남 신세계에서 밥먹을만한 곳 알려줘'}))

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '강남 신세계에서 밥먹을만한 곳 알려줘',
 'request_id': '694d0b13-1f47-4642-89a5-21fb8b163b47',
 'response_time': 1.27,
 'results': [{'content': '강남신세계 혼밥맛집 (106곳) · 1. 소녀방앗간 서울고속터미널점 · 2. 타코벨 '
                         '서울고속버스터미널점 · 3. 소이연남 파미에스테이션점 · 4. 갓덴스시 센트럴시티점 · 5. '
                         '101번지',
              'raw_content': None,
              'score': 0.72231215,
              'title': "'강남신세계 혼밥' 맛집 빅데이터 추천순위 Top100 - 다이닝코드",
              'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EC%8B%A0%EC%84%B8%EA%B3%84%20%ED%98%BC%EB%B0%A5'},
             {'content': 'Missing: 밥 알려줘',
              'raw_content': None,
              'score': 0.71948266,
              'title': '강남 신세계 푸드코트 하우스오브 신세계,스위트파크(Eng CC)/ Seoul ...',
              'url': 'https://www.youtube.com/watch?v=wRvUnu8OwWI'},
             {'content': 'Missing: 밥 알려줘',
              'raw_content': None,
              'score': 0.63316137,


In [18]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template="""넌 웹 검색결과를 요약해서 사용자의 질문에 맞게 좋은 답을 해주는 챗봇이야.

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 이 자리에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']} ({r['content']})' for r in x['results']]
            )
        )
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

"강남 신세계백화점에서 점심 식사로 추천할 만한 곳으로는 타치바나(지하 1층 하우스오브신세계)가 있으며, 인기 맛집으로는 '포비 강남점', '베통', '김수사' 등이 있습니다. 신세계백화점 내부에서는 미쉐린 빕 구르망 셰프의 점심 한정 메뉴도 경험해보실 수 있습니다."

In [17]:
dumb_chain = prompt | llm | StrOutputParser()

dumb_chain.invoke({'query': '강남 신세계에서 점심먹기 좋은곳', 'search_results': ''})

'강남 신세계 내에서 점심하기 좋은 곳으로는 다양한 레스토랑이 있습니다. 특히, 유명한 일식, 한식, 양식 레스토랑들이 있어 선택의 폭이 넓습니다. 추천드릴 만한 곳은 신세계백화점 내부의 푸드코트와 레스토랑가에서 맛있는 점심 식사를 하실 수 있습니다.'

In [26]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from datetime import datetime

today = datetime.today().strftime('%D')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

prompt = ChatPromptTemplate.from_messages([
    ('system', f'너는 훌륭한 어시스턴트야. 반드시 웹 검색도구를 사용해서 정보를 얻어. 최대한 답을 잘 해보자. 오늘은 {today}야.'),
    MessagesPlaceholder(variable_name='messages'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[search_tool], verbose=True)

agent_executor.invoke({'messages': [{'role': 'human', 'content': '오늘 날씨에 맞는 음악 추천해줘'}]})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '오늘의 한국 날씨', 'search_depth': 'basic'}`


{'query': '오늘의 한국 날씨', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.accuweather.com/ko/kr/seoul/226081/hourly-weather-forecast/226081', 'title': '서울특별시, 서울시, 대한민국 시간별 날씨', 'content': '오늘 ## 시간별 일별 레이더 MinuteCast® 대기질 건강 및 활동 ## 오후 9시 85° RealFeel® 96° 더움 대체로 맑음 경보 폭염 경보 경보 폭염 주의보 바람서북서 4mi/h 대기질보통 돌풍9mi/h 습도78% 이슬점77° F AccuLumen Brightness Index™0 (어둡게) 구름량12% 가시거리7mi 운저33500ft ## 오후 10시 83° RealFeel® 95° 더움 맑음 경보 폭염 경보 경보 폭염 주의보 바람서북서 4mi/h 대기질보통 돌풍7mi/h 습도82% 이슬점77° F AccuLumen Brightness Index™0 (어둡게) 구름량0% 가시거리7mi 운저30000ft ## 오후 11시 82° RealFeel® 94° 더움 맑음 경보 폭염 경보 경보 폭염 주의보 바람서북서 2mi/h 대기질보통 돌풍7mi/h 습도85% 이슬점77° F AccuLumen Brightness Index™0 (어둡게) 구름량0% 가시거리7mi 운저30000ft ### 내일 ### 월요일 ### 일별 전 세계 ### 허리케인 ### 악천후 기상 ### 레이더 및 지도 ### 동영상 ## 인근 지역: * 동작구, 서울시', 'score': 0.64639384, 'raw_content': None}, {'url':

{'messages': [{'role': 'human', 'content': '오늘 날씨에 맞는 음악 추천해줘'}],
 'output': '오늘 서울의 날씨는 맑고 쾌적하며, 낮 기온은 21도 정도로 예상됩니다. 이런 맑고 상쾌한 날씨에 어울리는 음악 추천 드릴게요. 최근 인기가 많고 기분 좋은 노래들입니다:\n\n1. **Surfaces - Good Day**  \n   밝고 경쾌한 멜로디로 하루를 긍정적으로 시작하게 하는 곡입니다.\n\n2. **Outkast - Hey Ya!**  \n   활기차면서도 기분 좋은 느낌의 팝/R&B곡으로, 산뜻한 하루에 딱 어울립니다.\n\n3. **Chordettes - Mr. Sandman**  \n   고전적인 분위기와 상큼한 느낌이 어우러져서 기분을 더욱 좋게 만들어주는 곡입니다.\n\n4. **김지수 - 오늘도 좋은 하루** (추천드릴 수 있는 한국 노래도 있어요)  \n   맑고 기분 좋은 느낌을 주는 노래로, 하루를 활기차게 만들어 줄 수 있어요.\n\n이 음악들을 들으며 산책하거나 커피 한잔의 여유를 즐기면 오늘 하루가 더욱 특별하게 느껴질 거예요! 어떤 스타일이 더 끌리시는지 말씀해주시면 더 맞춤형 추천도 가능합니다.'}